___
<img style="float: right; margin: 0px 0px 15px 15px;" 
src="https://www.carrerasenlinea.mx/logos/original/logo-instituto-tecnologico-y-de-estudios
-superiores-de-occidente.png" width="150px" height="100px" />


# MICROESTRUCTURAS Y SISTEMAS DE "TRADING"
# LABORATORIO 4

## Nombres: Ana Laura Correa , Mariana Lopez, Jocelyn Medel

## Fecha: 8 / Diciembre / 2019

#### Propósito: 
Diseñar una estrategia de trading con base a la reacción del precio ante el comunicado de indicadores macroeconómicos (Análisis fundamental).

#### Librerias:

In [1]:
#Importar librerias
import pandas as pd
pd.options.display.max_rows = None
import math 
import pandas as pd
import numpy as np
import warnings
import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.filterwarnings("ignore")

#### Archivo de Precios Historicos

In [2]:
prices = pd.read_csv('precios_historicos_eurusd.csv',header=0, index_col=None, parse_dates=False)
prices['timestamp'] = [pd.to_datetime(prices.iat[i,0]).tz_localize('GMT') for i in range(len(prices))] # Timestamp a 'GMT'
prices.head()

,timestamp,open,high,low,close
0,2009-06-01 05:00:00+00:00,1.35564,1.35565,1.35383,1.35439
1,2009-06-01 05:05:00+00:00,1.35443,1.35489,1.35424,1.35474
2,2009-06-01 05:10:00+00:00,1.35480,1.35552,1.35439,1.35539
3,2009-06-01 05:15:00+00:00,1.35545,1.35549,1.35101,1.35101
4,2009-06-01 05:20:00+00:00,1.35101,1.35130,1.35021,1.35086


#### Archivo de Calendario Economico

In [3]:
data = pd.read_csv('calendario_economico.csv',header=0, index_col=None, parse_dates=False)
data.head()

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus
0,1/5/09 9:30,EUR,Sentix Investor Confidence,European Monetary Union,1,-34.4,-42.3,NaN
1,1/5/09 15:00,USD,Construction Spending (MoM),United States,1,-1.2,-0.4,-1.3
2,1/6/09 8:55,EUR,Markit Services PMI,Germany,1,46.6,46.4,46.4
3,1/6/09 9:00,EUR,Markit Services PMI,European Monetary Union,0,42.1,42.0,42.0
4,1/6/09 10:00,EUR,Consumer Price Index (YoY),European Monetary Union,2,1.6,2.1,1.8


In [85]:
Nonfarm = data.loc[data["Name"] == "Nonfarm Payrolls"].reset_index(drop=True) #Resetear el indice, para encontrar calendario económico
Nonfarm.consensus.fillna(Nonfarm.previous, inplace=True) #Formar Dataframe
Nonfarm["timestamp"] = pd.to_datetime(Nonfarm["timestamp"])
Nonfarm["timestamp"] = Nonfarm["timestamp"].dt.tz_localize('GMT')
Nonfarm= Nonfarm.reset_index(drop= True)

### Escenarios del indicador

$A$ = $Actual$ ≥ $Consensus$ ≥ $Previous$

$B$ = $Actual$ ≥ $Consensus$ ≤ $Previous$

$C$ = $Actual$ ≤ $Consensus$ ≥ $Previous$

$D$ = $Actual$ ≤ $Consensus$ ≤ $Previous$

Donde:

$Actual$ : Es el valor actual del indicador.

$Consensus$ : Es el valor del indicador que se pronóstica.

$Previous$ : Es el valor anterior del indicador.

In [5]:
#Función  para encontrar los escenarios

def clase(x):
    """Param x:DataFrame
       Return x:DataFrame con columna de clase"""
    
    x["escenario"] = x["actual"]
    for i in range(len(x["actual"])):
        if x["actual"][i] >= x["consensus"][i] >= x["previous"][i]:
            x["escenario"][i] = "A"
        elif x["actual"][i] >= x["consensus"][i] < x["previous"][i]:
            x["escenario"][i] = "B"
        elif x["actual"][i] < x["consensus"][i] >= x["previous"][i]:
            x["escenario"][i] = "C"
        elif x["actual"][i] < x["consensus"][i] < x["previous"][i]:
            x["escenario"][i] = "D"
    
    return x

In [6]:
#Función para los periodos de 30 minutos antes y después de que se comunicó el índice 

def periodos(x,prices):
    xt = prices["timestamp"].isin(x["timestamp"])
    xt = [i for i in range(len(xt)) if xt[i] == True]
    t = [prices["timestamp"][i] for i in xt]
    xtprt = [prices.iloc[i-6:i+7] for i in xt]
    
    return xtprt,t

In [7]:
# Función para calcular las metricas señaladas 

def metrica(x,y,z):
    dire = lambda x: x.iloc[-1,-1]-x.iloc[int(len(x)/2),1]
    alc = lambda x: x.iloc[int(len(x)/2):,1].max()-x.iloc[int(len(x)/2),1]
    baj = lambda x: x.iloc[int(len(x)/2),1] - x.iloc[int(len(x)/2):,1].min()
    vol = lambda x: x["high"].max() - x["low"].min()
    
    direccion = [dire(i.reset_index(drop=True)) for i in y]
    alcista = [alc(i.reset_index(drop=True)) for i in y]
    bajista = [baj(i.reset_index(drop=True)) for i in y]
    volatilidad = [vol(i) for i in y]
    x = x[x["timestamp"].isin(z)]
    x["direccion"] = direccion
    x["alcista"] = alcista
    x["bajista"] = bajista
    x["volatilidad"] = volatilidad
    
    return x

In [8]:
#Aplicamos las funciones ya explicadas 

Nonfarm = clase(Nonfarm)

In [9]:
Nonfarmper,Nonfarmt = periodos(Nonfarm,prices)

In [10]:
Nonfarm = metrica(Nonfarm,Nonfarmper,Nonfarmt)

In [11]:
Nonfarm.head()

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus,escenario,direccion,alcista,bajista,volatilidad
0,2009-01-09 13:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-577.0,-584.0,-480.0,C,-0.00006,0.00082,0.00021,0.00419
1,2009-02-06 13:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-655.0,-577.0,-500.0,C,0.00120,0.00038,0.00036,0.00366
2,2009-03-06 13:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-651.0,-655.0,-625.0,C,-0.00329,0.00000,0.00168,0.00559
3,2009-04-03 12:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-699.0,-651.0,-658.0,D,0.00012,0.00125,0.00000,0.00636
4,2009-05-08 12:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-504.0,-699.0,-620.0,A,0.00487,0.00383,0.00070,0.00668


In [47]:
def escenarios(x,y,z):
    dire = lambda x: 1 if x.iloc[int(len(x)/2)]["close"] <= x.iloc[int(len(x)/2)]["open"]  else -1 
    alc = lambda x: (x.iloc[int(len(x)/2):,1].max()-x.iloc[int(len(x)/2),1])*10000
    baj = lambda x: (x.iloc[int(len(x)/2),1] - x.iloc[int(len(x)/2):,1].min())*10000
    vol = lambda x: x["high"].max() - x["low"].min()
    
    direccion = [dire(i.reset_index(drop=True)) for i in y]
    alcista = [alc(i.reset_index(drop=True)) for i in y]
    bajista = [baj(i.reset_index(drop=True)) for i in y]
    volatilidad = [vol(i) for i in y]
    
    x = x[x["timestamp"].isin(z)]
    x["direccion"] = direccion
    x["alcista"] = alcista
    x["bajista"] = bajista
    x["volatilidad"] = volatilidad
    


    return x

In [57]:
df = escenarios(Nonfarm,Nonfarmper,Nonfarmt)
df_escenarios= df[["timestamp","escenario","direccion","alcista", "bajista", "volatilidad" ]].reset_index(drop= True)
df_escenarios.head()

,timestamp,escenario,direccion,alcista,bajista,volatilidad
0,2009-01-09 13:30:00+00:00,C,1,8.2,2.1,0.00419
1,2009-02-06 13:30:00+00:00,C,1,3.8,3.6,0.00366
2,2009-03-06 13:30:00+00:00,C,1,0.0,16.8,0.00559
3,2009-04-03 12:30:00+00:00,D,-1,12.5,0.0,0.00636
4,2009-05-08 12:30:00+00:00,A,1,38.3,7.0,0.00668


In [28]:
A_count= list(df["escenario"]).count("A")
B_count= list(df["escenario"]).count("B")
C_count= list(df["escenario"]).count("C")
D_count= list(df["escenario"]).count("D")
print('The count of A is:', A_count)
print('The count of B is:', B_count)
print('The count of C is:', C_count)
print('The count of D is:', D_count)

The count of A is: 24
The count of B is: 24
The count of C is: 19
The count of D is: 20


In [29]:
A = df.loc[:,"escenario"] == 'A'
df_A = df.loc[A]
df_A.head()

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus,escenario,direccion,alcista,bajista,volatilidad
4,2009-05-08 12:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-504.0,-699.0,-620.0,A,1,38.3,7.0,0.00668
7,2009-08-07 12:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-276.0,-467.0,-345.0,A,1,18.3,6.8,0.00704
8,2009-09-04 12:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-201.0,-276.0,-223.0,A,-1,4.6,0.4,0.00344
10,2009-11-06 13:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-111.0,-219.0,-175.0,A,-1,28.6,0.0,0.00464
15,2010-04-02 12:30:00+00:00,USD,Nonfarm Payrolls,United States,3,230.0,-14.0,187.0,A,-1,10.2,0.0,0.00180


In [30]:
a=list(df_A["direccion"]).count(1)
promalc_A_m= (df_A["alcista"].min())*1.15
promalc_A_M= (df_A["alcista"].max())*.85
prombaj_A_m= (df_A["bajista"].min())*1.15
prombaj_A_M= (df_A["bajista"].max())*.85

In [31]:
B = df.loc[:,"escenario"] == 'B'
df_B = df.loc[B]
df_B.head()

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus,escenario,direccion,alcista,bajista,volatilidad
5,2009-06-05 12:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-322.0,-504.0,-521.0,B,-1,13.6,25.8,0.00498
14,2010-03-05 13:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-14.0,-26.0,-30.0,B,-1,9.4,0.0,0.00200
16,2010-05-07 12:30:00+00:00,USD,Nonfarm Payrolls,United States,3,290.0,230.0,177.0,B,1,0.0,23.0,0.00342
20,2010-09-03 12:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-57.0,-54.0,-105.0,B,-1,13.2,0.0,0.00382
27,2011-04-01 12:30:00+00:00,USD,Nonfarm Payrolls,United States,3,221.0,194.0,190.0,B,-1,8.7,3.6,0.00220


In [32]:
b=list(df_B["direccion"]).count(1)
promalc_B_m= (df_B["alcista"].min())*1.15
promalc_B_M= (df_B["alcista"].max())*.85
prombaj_B_m= (df_B["bajista"].min())*1.15
prombaj_B_M= (df_B["bajista"].max())*.85
b

13

In [33]:
C = df.loc[:,"escenario"] == 'C'
df_C = df.loc[C]
df_C.head()

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus,escenario,direccion,alcista,bajista,volatilidad
0,2009-01-09 13:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-577.0,-584.0,-480.0,C,1,8.2,2.1,0.00419
1,2009-02-06 13:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-655.0,-577.0,-500.0,C,1,3.8,3.6,0.00366
2,2009-03-06 13:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-651.0,-655.0,-625.0,C,1,0.0,16.8,0.00559
9,2009-10-02 12:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-219.0,-201.0,-188.0,C,1,2.8,5.0,0.00356
17,2010-06-04 12:30:00+00:00,USD,Nonfarm Payrolls,United States,3,433.0,290.0,502.0,C,1,0.0,25.2,0.00510


In [34]:
c=list(df_C["direccion"]).count(1)
promalc_C_m= (df_C["alcista"].min())*1.15
promalc_C_M= (df_C["alcista"].max())*.85
prombaj_C_m= (df_C["bajista"].min())*1.15
prombaj_C_M= (df_C["bajista"].max())*.85


In [35]:
D = df.loc[:,"escenario"] == 'D'
df_D = df.loc[D]
df_D.head()

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus,escenario,direccion,alcista,bajista,volatilidad
3,2009-04-03 12:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-699.0,-651.0,-658.0,D,-1,12.5,0.0,0.00636
23,2010-12-03 13:30:00+00:00,USD,Nonfarm Payrolls,United States,3,71.0,172.0,146.0,D,-1,8.1,8.5,0.00406
39,2012-04-06 12:30:00+00:00,USD,Nonfarm Payrolls,United States,3,154.0,240.0,203.0,D,1,0.0,16.5,0.00419
49,2013-02-01 13:30:00+00:00,USD,Nonfarm Payrolls,United States,3,119.0,196.0,160.0,D,1,0.0,23.1,0.00482
55,2013-08-02 12:30:00+00:00,USD,Nonfarm Payrolls,United States,3,104.0,188.0,184.0,D,-1,7.0,0.0,0.00161


In [37]:
d=list(df_D["direccion"]).count(-1)
promalc_D_m= (df_D["alcista"].min())*1.15
promalc_D_M= (df_D["alcista"].max())*.85
prombaj_D_m= (df_D["bajista"].min())*1.15
prombaj_D_M= (df_D["bajista"].max())*.85


In [76]:
def operacion(x):
    """Param x:DataFrame
       Return x:DataFrame con columna de clase"""

    x["operacion"] = x["escenario"]
    x["sl"] = x["escenario"]
    x["tp"] = x["escenario"]
    x["volumen"]= x["escenario"]

    for i in range(len(x["escenario"])):
        if x["escenario"][i] == "A" :
            x["operacion"][i] = "Compra"
            x["sl"][i]=  round((prombaj_A_m + prombaj_A_M) / 2 )
            x["tp"][i]=  round((promalc_A_m + promalc_A_M) / 2 )
            x["volumen"][i]= 135500
        elif x["escenario"][i] == "B":
            x["operacion"][i] = "Compra"
            x["sl"][i]=  round((prombaj_B_m + prombaj_B_M) / 2 )
            x["tp"][i]=  round((promalc_B_m + promalc_B_M) / 2 )
            x["volumen"][i]=  232500
        elif x["escenario"][i] == "C":
            x["operacion"][i] = "Compra"
            x["sl"][i]=  round((prombaj_C_m + prombaj_C_M) / 2 )
            x["tp"][i]=  round((promalc_C_m + promalc_C_M) / 2 )
            x["volumen"][i]= 232500
        elif x["escenario"][i] == "D":
            x["operacion"][i] = "Venta"
            x["sl"][i]=  round((promalc_D_m + promalc_D_M) / 2 )
            x["tp"][i]=  round((prombaj_D_m + prombaj_D_M) / 2 )
            x["volumen"][i]= 185000
    return x

In [77]:
df_dec= operacion(df_escenarios)

In [78]:
df_decisiones= df_dec[["escenario", "operacion", "sl", "tp", "volumen"]].drop_duplicates().reset_index(drop=True)
df_decisiones.head()

,escenario,operacion,sl,tp,volumen
0,C,Compra,11,19,232500
1,D,Venta,13,23,185000
2,A,Compra,11,25,135500
3,B,Compra,16,25,232500


In [91]:
def calculo(x,y):
    
    x["resultado"]= x["volumen"]
    x["pips"]= x["volumen"]
    x["capital"]= x["volumen"] 
    
    capital= 100000 
    
    for i in range(len(x)):
        m = y[i]
       
        for j in range(len(m.iloc[6:])):
            
            if x["operacion"][i] == "Compra" :
            
                if ((x["tp"][i])/10000) + m.iloc[int(len(m)/2)]["open"] >= m.iloc[j]["high"]  or ((x["sl"][i])/10000) - m.iloc[int(len(m)/2)]["open"] <= m.iloc[j]["low"]  :
                    if ((x["tp"][i])/10000) + m.iloc[int(len(m)/2)]["open"] >=  m.iloc[j]["high"] :
                        a=   m.iloc[int(len(m)/2)]["open"]                  
                        b= ((x["tp"][i])/10000) + m.iloc[int(len(m)/2)]["open"] 
                        c= x["volumen"][i]*b - x["volumen"][i]*a
                        x["resultado"][i] = "ganadora" if c > 0  else "perdedora"
                        x["pips"][i]= (b-a)*10000
                        x["capital"][i] = capital + c 
                    else :
                        a= m.iloc[int(len(m)/2)]["open"]
                        b= m.iloc[int(len(m)/2)]["open"]- ((x["sl"][i])/10000)  
                        c= x["volumen"][i]* a - x["volumen"][i]* b 
                        x["resultado"][i] = "ganadora" if c > 0  else "perdedora"
                        x["pips"][i]= (a-b)/10000
                        x["capital"][i] = capital + c                         
                else :
                    a= m.iloc[int(len(m)/2)]["open"]   
                    b= m.iloc[int(len(m)/2)]["close"] 
                    c= x["volumen"][i]*a  - x["volumen"][i]*b
                    x["resultado"][i] = "ganadora" if c > 0  else "perdedora"
                    x["pips"][i]= (a-b)*10000
                    x["capital"][i] = capital + c       
            
            elif x["operacion"][i] == "Venta" : 
            
                if (( m.iloc[int(len(m)/2)]["open"] - x["tp"][i])/10000)  >=  m.iloc[j]["high"]  or  ((x["sl"][i])/10000) + m.iloc[int(len(m)/2)]["open"]  <= m.iloc[j]["low"] :
                    if  ((x["tp"][i])/10000) - m.iloc[int(len(m)/2)]["open"] >= m.iloc[j]["high"]  :
                        a= m.iloc[int(len(m)/2)]["open"]                  
                        b=  m.iloc[int(len(m)/2)]["open"] - ((x["tp"][i])/10000)  
                        c= x["volumen"][i]*a - x["volumen"][i]*a
                        x["resultado"][i] = "ganadora" if c > 0  else "perdedora"
                        x["pips"][i]= (b-a)*10000
                        x["capital"][i] = capital + c 
                    else :
                        a= m.iloc[int(len(m)/2)]["open"] 
                        b=((x["sl"][i])/10000) + m.iloc[int(len(m)/2)]["open"] 
                        c= x["volumen"][i]* a - x["volumen"][i]*b
                        x["resultado"][i] = "ganadora" if c > 0  else "perdedora"
                        x["pips"][i]= (a-b)*10000
                        x["capital"][i] = capital + c                         
                else :
                    
                    a= m.iloc[int(len(m)/2)]["open"]   
                    b= m.iloc[int(len(m)/2)]["close"] 
                    c= x["volumen"][i]*a - x["volumen"][i]*b
                    x["resultado"][i] = "ganadora" if c > 0  else "perdedora"
                    x["pips"][i]= (a-b)*10000
                    x["capital"][i] = capital + c       

  
    return x            

In [92]:
df_backtest= calculo(df_dec,Nonfarmper)
df_backtest= df_backtest[["escenario", "operacion","volumen", "resultado", "pips", "capital"]]



In [102]:
df_backtest.head()

,timestamp,escenario,direccion,alcista,bajista,volatilidad,operacion,sl,tp,volumen,resultado,pips,capital
0,2009-01-09 13:30:00+00:00,C,1,8.2,2.1,0.00419,Compra,11,19,232500,ganadora,19,100442
1,2009-02-06 13:30:00+00:00,C,1,3.8,3.6,0.00366,Compra,11,19,232500,ganadora,19,100442
2,2009-03-06 13:30:00+00:00,C,1,0.0,16.8,0.00559,Compra,11,19,232500,ganadora,19,100442
3,2009-04-03 12:30:00+00:00,D,-1,12.5,0.0,0.00636,Venta,13,23,185000,perdedora,-3.8,99929.7
4,2009-05-08 12:30:00+00:00,A,1,38.3,7.0,0.00668,Compra,11,25,135500,ganadora,25,100339


### Prueba

In [126]:
prices_2 = pd.read_csv('precios_historicos_eurusd_2019.csv',header=0, index_col=None, parse_dates=False)
prices_2['timestamp'] = [pd.to_datetime(prices_2.iat[i,0]).tz_localize('GMT') for i in range(len(prices_2))] # Timestamp a 'GMT'


In [127]:
Nonfarm_2 = data.loc[data["Name"] == "Nonfarm Payrolls"].reset_index(drop=True) #Resetear el indice, para encontrar calendario económico
Nonfarm_2.consensus.fillna(Nonfarm_2.previous, inplace=True) #Formar Dataframe
Nonfarm_2["timestamp"] = pd.to_datetime(Nonfarm_2["timestamp"])
Nonfarm_2["timestamp"] = Nonfarm_2["timestamp"].dt.tz_localize('GMT')
Nonfarm_2= Nonfarm_2.reset_index(drop= True)

In [128]:
Nonfarm_2 = clase(Nonfarm_2)
Nonfarmper_2,Nonfarmt_2 = periodos(Nonfarm_2 ,prices_2)
Nonfarm_2 = metrica(Nonfarm_2 ,Nonfarmper_2,Nonfarmt_2)

In [129]:
df_2 = escenarios(Nonfarm_2,Nonfarmper_2,Nonfarmt_2)
df_escenarios_2= df_2[["timestamp","escenario","direccion","alcista", "bajista", "volatilidad" ]]
df_escenarios_2 = df_escenarios_2.reset_index(drop=True)

In [130]:
df_dec_2= operacion(df_escenarios_2)
df_decisiones_2= df_dec_2[["timestamp","escenario", "operacion", "sl", "tp", "volumen"]].drop_duplicates().reset_index(drop=True)

In [131]:
df_prueba= calculo(df_dec_2,Nonfarmper_2)

In [132]:
df_prueba= df_prueba[["timestamp","escenario", "operacion","volumen", "resultado", "pips", "capital"]]
df_prueba

,timestamp,escenario,operacion,volumen,resultado,pips,capital
0,2019-01-04 13:30:00+00:00,A,Compra,135500,ganadora,25,100339
1,2019-02-01 13:30:00+00:00,B,Compra,232500,ganadora,25,100581
2,2019-03-08 13:30:00+00:00,D,Venta,185000,ganadora,4.8,100089
3,2019-04-05 12:30:00+00:00,A,Compra,135500,ganadora,25,100339
4,2019-05-03 12:30:00+00:00,B,Compra,232500,ganadora,25,100581
5,2019-06-07 12:30:00+00:00,D,Venta,185000,perdedora,-35.4,99345.1
6,2019-07-05 12:30:00+00:00,A,Compra,135500,ganadora,25,100339
7,2019-08-02 12:30:00+00:00,B,Compra,232500,ganadora,25,100581
8,2019-09-06 12:30:00+00:00,D,Venta,185000,ganadora,4.3,100080
